In [1]:
import EOSutils as meos

import numpy as np
import matplotlib.pyplot as plt

import matplotlib.style
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable

mpl.style.use('classic')

from astropy.table import Table
from astropy import units as u
from astropy.constants import G

import mesa_helper as mh
import os
import shutil
import copy

from scipy import interpolate
from scipy.linalg import lstsq as scipy_lstsq
from scipy.ndimage import gaussian_filter
import scipy.integrate as integrate
import decimal 

%matplotlib inline

In [2]:
He_mol_mass = 4.0026 #g/mol
H2_mol_mass = 2*1.008 #g/mol
D2_mol_mass = 2*2.0141 #g/mol

# He

In [8]:
# needing conversion: Nellis+1984
nellis1984 = Table.read("./He_nellis+1984.dat",format="ascii",header_start=1)
print(nellis1984)

rho = 1./(nellis1984['V[cm3/mol]'] * (1./He_mol_mass))
rho_unc_lower = (He_mol_mass/nellis1984['V[cm3/mol]']**2) * nellis1984['V_unc_lower[cm3/mol]']
rho_unc_upper = (He_mol_mass/nellis1984['V[cm3/mol]']**2) * nellis1984['V_unc_lower[cm3/mol]']

nellis1984.add_column(np.round(rho,5),name='rho[g/cm3]')
nellis1984.add_column(np.round(rho_unc_lower,5),name='rho_unc_lower[g/cm3]')
nellis1984.add_column(np.round(rho_unc_upper,5),name='rho_unc_upper[g/cm3]')

eta = rho/nellis1984['rho_0[g/cm3]']
eta_unc_lower = rho_unc_lower/nellis1984['rho_0[g/cm3]']
eta_unc_upper = rho_unc_upper/nellis1984['rho_0[g/cm3]']

nellis1984.add_column(np.round(eta,5),name='eta[unitless]')
nellis1984.add_column(np.round(eta_unc_lower,5),name='eta_unc_lower[unitless]')
nellis1984.add_column(np.round(eta_unc_upper,5),name='eta_unc_upper[unitless]')


nellis1984.write("./He_nellis+1984.dat", format='ascii',overwrite=True)

rho_0[g/cm3] P[GPa] ... V_unc_lower[cm3/mol] V_unc_upper[cm3/mol]
------------ ------ ... -------------------- --------------------
      0.1233   1.23 ...                  0.4                  0.4
      0.1233    6.5 ...                  0.2                  0.2
      0.1235   15.6 ...                  0.3                  0.3


# H2

In [10]:
# needing conversion: Nellis+1983
nellis1983 = Table.read("./H2_nellis+1983.dat",format="ascii",header_start=1)
print(nellis1983)

rho = 1./(nellis1983['V[cm3/mol]'] * (1./H2_mol_mass))
rho_unc_lower = (H2_mol_mass/nellis1983['V[cm3/mol]']**2) * nellis1983['V_unc_lower[cm3/mol]']
rho_unc_upper = (H2_mol_mass/nellis1983['V[cm3/mol]']**2) * nellis1983['V_unc_lower[cm3/mol]']

nellis1983.add_column(np.round(rho,5),name='rho[g/cm3]')
nellis1983.add_column(np.round(rho_unc_lower,5),name='rho_unc_lower[g/cm3]')
nellis1983.add_column(np.round(rho_unc_upper,5),name='rho_unc_upper[g/cm3]')

eta = rho/nellis1983['rho_0[g/cm3]']
eta_unc_lower = rho_unc_lower/nellis1983['rho_0[g/cm3]']
eta_unc_upper = rho_unc_upper/nellis1983['rho_0[g/cm3]']

nellis1983.add_column(np.round(eta,5),name='eta[unitless]')
nellis1983.add_column(np.round(eta_unc_lower,5),name='eta_unc_lower[unitless]')
nellis1983.add_column(np.round(eta_unc_upper,5),name='eta_unc_upper[unitless]')


nellis1983.write("./H2_nellis+1983.dat", format='ascii',overwrite=True)

rho_0[g/cm3] P[GPa] ... V_unc_lower[cm3/mol] V_unc_upper[cm3/mol]
------------ ------ ... -------------------- --------------------
      0.0709   1.93 ...                 0.14                 0.14
      0.0706   4.11 ...                 0.18                 0.18
      0.0711   8.71 ...                 0.25                 0.25
       0.071  10.24 ...                 0.28                 0.28


# D2

In [16]:
# belov+2002
belov2002 = Table.read("./D2_belov+2002.dat",format="ascii",header_start=0)
print(belov2002)

eta  = belov2002['rho[g/cm3]']/belov2002['rho_0[g/cm3]']
eta_unc_lower = belov2002['rho_unc_lower[g/cm3]']/belov2002['rho_0[g/cm3]']
eta_unc_upper = belov2002['rho_unc_upper[g/cm3]']/belov2002['rho_0[g/cm3]']

belov2002.add_column(np.round(eta,5),name='eta[unitless]')
belov2002.add_column(np.round(eta_unc_lower,5),name='eta_unc_lower[unitless]')
belov2002.add_column(np.round(eta_unc_upper,5),name='eta_unc_upper[unitless]')

belov2002.write("./D2_belov+2002.dat",format="ascii",overwrite=True)

rho_0[g/cm3] rho[g/cm3] ... P_unc_lower[GPa] P_unc_upper[GPa]
------------ ---------- ... ---------------- ----------------
       0.199      0.774 ...                0                0


In [17]:
# boriskov+2003, 2005

boriskov2003 = Table.read("./D2_boriskov+2003.dat",format="ascii",header_start=0)
print(boriskov2003)

eta  = boriskov2003['rho[g/cm3]']/boriskov2003['rho_0[g/cm3]']
eta_unc_lower = boriskov2003['rho_unc_lower[g/cm3]']/boriskov2003['rho_0[g/cm3]']
eta_unc_upper = boriskov2003['rho_unc_upper[g/cm3]']/boriskov2003['rho_0[g/cm3]']

boriskov2003.add_column(np.round(eta,5),name='eta[unitless]')
boriskov2003.add_column(np.round(eta_unc_lower,5),name='eta_unc_lower[unitless]')
boriskov2003.add_column(np.round(eta_unc_upper,5),name='eta_unc_upper[unitless]')

boriskov2003.write("./D2_boriskov+2003.dat",format="ascii",overwrite=True)

boriskov2005 = Table.read("./D2_boriskov+2005.dat",format="ascii",header_start=0)
print(boriskov2005)

eta  = boriskov2005['rho[g/cm3]']/boriskov2005['rho_0[g/cm3]']
eta_unc_lower = boriskov2005['rho_unc_lower[g/cm3]']/boriskov2005['rho_0[g/cm3]']
eta_unc_upper = boriskov2005['rho_unc_upper[g/cm3]']/boriskov2005['rho_0[g/cm3]']

boriskov2005.add_column(np.round(eta,5),name='eta[unitless]')
boriskov2005.add_column(np.round(eta_unc_lower,5),name='eta_unc_lower[unitless]')
boriskov2005.add_column(np.round(eta_unc_upper,5),name='eta_unc_upper[unitless]')

boriskov2005.write("./D2_boriskov+2005.dat",format="ascii",overwrite=True)

rho_0[g/cm3] rho[g/cm3] ... P_unc_lower[GPa] P_unc_upper[GPa]
------------ ---------- ... ---------------- ----------------
       0.199       0.85 ...                0                0
rho_0[g/cm3] rho[g/cm3] ... P_unc_lower[GPa] P_unc_upper[GPa]
------------ ---------- ... ---------------- ----------------
       0.171      0.608 ...              0.8              0.8
       0.171      0.697 ...              0.6              0.6
       0.199      0.749 ...              0.8              0.8
       0.171      0.724 ...              3.0              3.0
       0.199      0.808 ...              2.0              2.0


In [20]:
# collins+1998
collins1998_rho0 = 0.17 #g/cm3

collins1998 = Table.read("./D2_collins+1998.dat",format="ascii",header_start=1)
print(collins1998)

eta  = collins1998['rho[g/cm3]']/collins1998_rho0
eta_unc_lower = collins1998['rho_unc_lower[g/cm3]']/collins1998_rho0
eta_unc_upper = collins1998['rho_unc_upper[g/cm3]']/collins1998_rho0

collins1998.add_column(np.round(eta,5),name='eta[unitless]')
collins1998.add_column(np.round(eta_unc_lower,5),name='eta_unc_lower[unitless]')
collins1998.add_column(np.round(eta_unc_upper,5),name='eta_unc_upper[unitless]')

collins1998.write("./D2_collins+1998.dat",format="ascii",overwrite=True)

rho[g/cm3] rho_unc_lower[g/cm3] ... P_unc_lower[GPa] P_unc_upper[GPa]
---------- -------------------- ... ---------------- ----------------
    0.4754                  0.0 ...              0.0          0.42103
   0.52473              0.01738 ...          0.48474          0.39782
   0.56292              0.01509 ...          0.50551          0.50586
   0.56859              0.02342 ...          1.85859           1.9564
   0.59414              0.05838 ...          1.70847          1.63163
   0.79817              0.15494 ...          2.37123          1.67087
   0.93913              0.06144 ...          1.09323          1.47146
   0.94234              0.16419 ...          3.43455          2.48398
   0.90124              0.06456 ...          5.13835          5.00613
   0.99273              0.11137 ...          4.75709          4.90346
   1.00136              0.04093 ...          5.12491          3.85194
   1.03048              0.15822 ...          4.50316          2.69201
   0.99867          

In [24]:
# grishechkin+2004

grishechkin2004 = Table.read("./D2_grishechkin+2004.dat",format="ascii",header_start=1)
print(grishechkin2004)

eta  = grishechkin2004['rho[g/cm3]']/grishechkin2004['rho_0[g/cm3]']
eta_unc_lower = grishechkin2004['rho_unc_lower[g/cm3]']/grishechkin2004['rho_0[g/cm3]']
eta_unc_upper = grishechkin2004['rho_unc_upper[g/cm3]']/grishechkin2004['rho_0[g/cm3]']

grishechkin2004.add_column(np.round(eta,5),name='eta[unitless]')
grishechkin2004.add_column(np.round(eta_unc_lower,5),name='eta_unc_lower[unitless]')
grishechkin2004.add_column(np.round(eta_unc_upper,5),name='eta_unc_upper[unitless]')

grishechkin2004.write("./D2_grishechkin+2004.dat",format="ascii",overwrite=True)


rho_0[g/cm3] rho[g/cm3] ... P_unc_lower[GPa] P_unc_upper[GPa]
------------ ---------- ... ---------------- ----------------
       0.134       0.64 ...              0.0              0.0
       0.153        0.7 ...              0.0              0.0
       0.171       0.77 ...              0.0              0.0


In [13]:
# nellis+1983

# needing conversion: Nellis+1983
nellis1983_D2 = Table.read("./D2_nellis+1983.dat",format="ascii",header_start=1)
print(nellis1983_D2)

rho = 1./(nellis1983_D2['V[cm3/mol]'] * (1./D2_mol_mass))
rho_unc_lower = (D2_mol_mass/nellis1983_D2['V[cm3/mol]']**2) * nellis1983_D2['V_unc_lower[cm3/mol]']
rho_unc_upper = (D2_mol_mass/nellis1983_D2['V[cm3/mol]']**2) * nellis1983_D2['V_unc_lower[cm3/mol]']

nellis1983_D2.add_column(np.round(rho,5),name='rho[g/cm3]')
nellis1983_D2.add_column(np.round(rho_unc_lower,5),name='rho_unc_lower[g/cm3]')
nellis1983_D2.add_column(np.round(rho_unc_upper,5),name='rho_unc_upper[g/cm3]')

eta = rho/nellis1983_D2['rho_0[g/cm3]']
eta_unc_lower = rho_unc_lower/nellis1983_D2['rho_0[g/cm3]']
eta_unc_upper = rho_unc_upper/nellis1983_D2['rho_0[g/cm3]']

nellis1983_D2.add_column(np.round(eta,5),name='eta[unitless]')
nellis1983_D2.add_column(np.round(eta_unc_lower,5),name='eta_unc_lower[unitless]')
nellis1983_D2.add_column(np.round(eta_unc_upper,5),name='eta_unc_upper[unitless]')


nellis1983_D2.write("./D2_nellis+1983.dat", format='ascii',overwrite=True)

rho_0[g/cm3] P[GPa] ... V_unc_lower[cm3/mol] V_unc_upper[cm3/mol]
------------ ------ ... -------------------- --------------------
      0.1708   4.06 ...                 0.14                 0.14
      0.1706   6.76 ...                0.197                0.197
      0.1704  10.35 ...                0.157                0.157
      0.1711  14.32 ...                0.288                0.288
      0.1707   19.9 ...                0.205                0.205
